In [61]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

<h2> Skills </h2>
Come prima parte della generazione dei dati, è necessario specificare quali siano le possibili skill che un developer possa avere.


In [62]:
with sqlite3.connect('../datasets/skills_dataset.db') as skills_conn:
    skills_frame = pd.read_sql('SELECT * FROM Skills', skills_conn, index_col='ID') 
skills_frame

,SKILL,TYPE
ID,,
0,JavaScript,Programming Language
1,HTML,Programming Language
2,Python,Programming Language
3,SQL,Programming Language
4,TypeScript,Programming Language
...,...,...
100,Git,Tool
101,Travis,Tool
102,Swift,Programming Language


In [63]:
frame = pd.read_csv('../datasets/datasetPULITO (1).csv', index_col = 'ID')
frame

,Linguaggi_di_Programmazione,Framework,Database,Tool,Cloud
ID,,,,,
1,"JavaScript,HTML,Python,CSS","Node.js,React,jQuery,Express","PostgreSQL,MySQL,SQLite","Docker,npm,Pip,Webpack",AWS
2,"TypeScript,Java,C#","Angular,Vue.js","Microsoft SQL Server,MySQL","Kubernetes,Maven,Docker",Google Cloud
3,"C++,C,Java",NaN,"PostgreSQL,SQLite","CMake,GCC",AWS
4,"HTML,JavaScript,PHP,CSS","Node.js, Vue.js",MySQL,"Composer,npm",Vercel
5,"Python,Java,Ruby,Go",Flask,"MongoDB,MySQL,Redis","Docker,Cargo","Google Cloud, Heroku"
6,Python,"Flask, FastAPI",MySQL,"npm,Docker",NaN
7,"Bash,PowerShell,C,Python",NaN,MySQL,Docker,Azure
8,"Go,Rust",NaN,"SQLite,MariaDB",Docker,AWS
9,"Ruby,JavaScript,HTML,CSS","Node.js,Next.js",MongoDB,"RubyGems,npm",Heroku


In [64]:
frame['skills'] = frame.apply(lambda row: ','.join(str(val) for val in row if not pd.isna(val)), axis = 1)
column_to_remove = ['Linguaggi_di_Programmazione','Framework','Database','Tool','Cloud']

frame = frame.drop(column_to_remove, axis = 1)


In [65]:
frame

,skills
ID,
1,"JavaScript,HTML,Python,CSS,Node.js,React,jQuer..."
2,"TypeScript,Java,C#,Angular,Vue.js,Microsoft SQ..."
3,"C++,C,Java,PostgreSQL,SQLite,CMake,GCC,AWS"
4,"HTML,JavaScript,PHP,CSS,Node.js, Vue.js,MySQL,..."
5,"Python,Java,Ruby,Go,Flask,MongoDB,MySQL,Redis,..."
6,"Python,Flask, FastAPI,MySQL,npm,Docker"
7,"Bash,PowerShell,C,Python,MySQL,Docker,Azure"
8,"Go,Rust,SQLite,MariaDB,Docker,AWS"
9,"Ruby,JavaScript,HTML,CSS,Node.js,Next.js,Mongo..."


In [66]:
actgan_generated_frame = pd.read_csv('../datasets/actgan_generated_dev_dataset.csv')

actgan_generated_frame

,ID,skills
0,1,"C++,Python,NumPy,Scikit-Learn,Tensorflow,Panda..."
1,37,"JavaScript,TypeScript,HTML,CSS,Svelte,MySQL,np..."
2,1,"Python,Flask, FastAPI,MySQL,npm,Docker,Python,..."
3,5,"Java,Kotlin,Spring,Oracle,Maven,Gradle,Google ..."
4,50,"Python,Java,Ruby,Go,Flask,Spring,MongoDB,MySQL..."
...,...,...
1495,28,"C++,Python,NumPy,Scikit-Learn,Tensorflow,Panda..."
1496,29,"Dart,JavaScript,TypeScript,HTML,CSS,Flutter,An..."
1497,5,"Python,Flask, FastAPI,MySQL,npm,Docker,Python,..."
1498,2,"TypeScript,Java,C#,Angular,Vue.js,Microsoft SQ..."


In [67]:
actgan_generated_frame = actgan_generated_frame.drop("ID", axis = 1 )

actgan_generated_frame

,skills
0,"C++,Python,NumPy,Scikit-Learn,Tensorflow,Panda..."
1,"JavaScript,TypeScript,HTML,CSS,Svelte,MySQL,np..."
2,"Python,Flask, FastAPI,MySQL,npm,Docker,Python,..."
3,"Java,Kotlin,Spring,Oracle,Maven,Gradle,Google ..."
4,"Python,Java,Ruby,Go,Flask,Spring,MongoDB,MySQL..."
...,...
1495,"C++,Python,NumPy,Scikit-Learn,Tensorflow,Panda..."
1496,"Dart,JavaScript,TypeScript,HTML,CSS,Flutter,An..."
1497,"Python,Flask, FastAPI,MySQL,npm,Docker,Python,..."
1498,"TypeScript,Java,C#,Angular,Vue.js,Microsoft SQ..."


In [69]:
gpt_generated_frame = pd.read_csv('../datasets/gpt_generated_dev_dataset.csv')

gpt_generated_frame

,skills
0,"Java,Kotlin,Python,Rust,JakartaEE,Spring,MySQL..."
1,"JavaScript,HTML,Python,CSS,Node.js,jQuery,Djan..."
2,"C++,C,Python,Java,Spring,MySQL,Maven,Gradle,C+..."
3,"HTML,JavaScript,TypeScript, CSS,Angular,React,..."
4,"C++,Python,NumPy,Scikit-Learn,Tensorflow,Panda..."
...,...
1495,"Go,Rust,Kotlin,Python,FastAPI,SQLite,MariaDB,D..."
1496,"HTML,JavaScript,PHP,CSS,Node.js, Vue.js,MySQL,..."
1497,"Java,JavaScript,Spring, JakartaEE,OracleDB,Pos..."
1498,"JavaScript,TypeScript,HTML, CSS,Svelte,React,M..."
